In [26]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()
import os
from supabase import create_client
import pandas as pd
import datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta
import logging
import numpy as np
from bs4 import BeautifulSoup

In [2]:
url_supabase = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")
supabase = create_client(url_supabase, key)

In [48]:
data_code = supabase.table("idx_company_profile").select("*").execute()
data = pd.DataFrame(data_code.data)

# Get Investing Links

In [49]:
symbols = [ticker.lower().replace(".jk", "") for ticker in data["symbol"][data["current_source"] == 2].tolist()]
avail_data =  [ 'bbca', 'amar', 'maba', 'cowl', 'btps', 'agrs', 'agro', 'life','bmas', 'bvic', 'mega', 'bsim', 'arto', 
                'mrei', 'asrm', 'bmri', 'bbri', 'home', 'bpfi', 'smil', 'lpgi', 'bbkp', 'bris', 'bina', 'tril', 'forz',
                'inet', 'bank', 'krah', 'dnar', 'amag', 'bcic', 'dcii', 'hill', 'plas', 'beks', 'hatm', 'pnbs', 'bbhi', 
                'nips', 'irsx', 'mcor', 'bbtn', 'bgtg', 'maya', 'bhat', 'nisp', 'nobu', 'goll', 'bnga',  'magp', 'npgf',
                'imas', 'pnbn', 'bswd', 'pnin', 'bbyb', 'bjtm', 'babp', 'bbmd', 'abda', 'admf', 'kbri', 'jsky', 'baca', 
                'sdra', 'miti', 'tram', 'buah', 'btpn', 'bksw', 'bnba', 'bbsi', 'cuan', 'bnli', 'gsmf', 'asdm', 'casa', 
                'bdmn', 'pnlf', 'nusa', 'beef', 'skyb', 'sugi', 'smma', 'asmi', 'tugu', 'myrx', 'bbni', 'inpc', 'bnii', 
                'bjbr', 'hotl', 'army', 'duck', 'lcgp']
na = [symbol for symbol in symbols if symbol not in avail_data]
na_list = [miss.upper() + ".JK" for miss in na]

In [50]:
data = data[data["symbol"].isin(na_list)]
companies = [company.replace("pt ", "").replace(" tbk", "").replace(" ", "-").replace(".", "").strip() for company in data["company_name"].str.lower()]
symbols = data["symbol"].tolist()

In [56]:
dict_na = {
    "symbols" : [],
    "href" : []
}
for stock in na:
    link = f"https://www.investing.com/search/?q={stock}"
    content = requests.get(link).text
    soup = BeautifulSoup(content, 'html.parser')
    search_results = soup.find_all(class_ = "js-inner-all-results-quote-item row")
    for search_result in search_results:
        text = search_result.get_text()
        if "Stock - Jakarta" in text:
            ticker = search_result.find("span", class_ = "second")
            if ticker.get_text().lower() == stock:
                dict_na["symbols"].append(stock)
                dict_na["href"].append(search_result["href"])

In [58]:
data_link = pd.DataFrame(dict_na)
data_link = data_link[data_link["symbols"] != "asha"]
leftover = {
    "unit" : "/equities/nusantara-inti",
    "trus" : "/equities/trust-finance-indonesia-tbk-pt",
    "gold" : "/equities/golden-retail",
    "euro" : "/equities/estee-gold-feet-pt-tbk",
    "digi" : "/equities/arkadia-digital",
    "food"  : "/equities/sentra-food",
    "glob" : "/equities/global-telesho",
    "real" : "/equities/repower-asia-indonesia-pt",
    "asha" : "/equities/cilacap-samudera-fishing-industry",
    "cntx" : "/equities/centex-tbk-sp"
}
lft = {
    "symbols" : [],
    "href" : []
}
for key, value in zip(leftover.keys(), leftover.values()):
    lft["symbols"].append(key)
    lft["href"].append(value)
lft = pd.DataFrame(lft)
data_link = pd.concat([data_link, lft])

In [60]:
extension = {
    "symbols" : [],
    "id" : [],
    "currency" : [],
    "fold" : []
}
for symbol, link in zip(data_link["symbols"], data_link["href"]):
    try:
        url = f"https://www.investing.com{link}-financial-summary"
        html_content = requests.get(url).text
        soup = BeautifulSoup(html_content, "html.parser")
        check = soup.find(class_="headBtnWrapper float_lang_base_2 js-add-alert-widget")
        id = check["data-pair-id"]
        info = soup.find(class_="arial_11 lightgrayFont bottomRemark").get_text()
        currency = "IDR" if "IDR" in info else "USD"
        fold = "Millions" if "Millions" in info else "Billions"
        extension["symbols"].append(symbol)
        extension["id"].append(id)
        extension["currency"].append(currency)
        extension["fold"].append(fold)
    except Exception as e:
        print(f"error pada link {link} dengan error: {e}")
        extension["symbols"].append(symbol)
        extension["id"].append(np.nan)
        extension["currency"].append(np.nan)
        extension["fold"].append(np.nan)

error pada link /equities/maxindo-karya-anugerah?cid=1203353 dengan error: 'NoneType' object has no attribute 'get_text'
error pada link /equities/pt-royaltama-mulia-kontraktorindo dengan error: 'NoneType' object has no attribute 'get_text'
error pada link /equities/graha-mitra-asia-tbk-pt dengan error: 'NoneType' object has no attribute 'get_text'


In [67]:
no_data = ["klas", "rmko", "relf"]
data_link_new = pd.merge(data_link, pd.DataFrame(extension), on = "symbols")
data_link_new = data_link_new[~data_link_new["symbols"].isin(no_data)].reset_index(drop = True)
data_link_new.to_csv("investing_link.csv", index = False)

In [68]:
status_dict = {
    "symbol" : [],
    "status" : [],
    "should_be_gross_margin" : []
}
failure = []
for symbol,company in zip(data_link_new["symbols"], data_link_new["href"]):
        link = f"https://www.investing.com{company}-financial-summary"
        status = str(requests.get(link))
        try:
            html_content = requests.get(link).text
            soup = BeautifulSoup(html_content, 'html.parser')
            values = soup.find("span", class_="float_lang_base_1")
            status_dict["should_be_gross_margin"].append(values.get_text())
            status_dict["symbol"].append(symbol.upper() + ".JK")
            status_dict["status"].append(status)
        except Exception as e:
            print(f"symbol {symbol} error: {e}")
            failure.append(symbol)

In [38]:
status_df = pd.DataFrame(status_dict)
[symbol for symbol in na_list if symbol not in status_df["symbol"][status_df["should_be_gross_margin"] == "Gross margin"].tolist()]

['KLAS.JK', 'RMKO.JK', 'RELF.JK']

3 symbol di atas tidak bisa discraping pada investing.com

# Scraper Initialization

In [69]:
data_link = pd.read_csv("investing_link.csv")

In [70]:
data_link

,symbols,href,id,currency,fold
0,cbre,/equities/cakra-buana-resources-energi-pt-tbk,1198683,IDR,Millions
1,laju,/equities/pt-jasa-berdikari-logistics-tbk,1199667,IDR,Millions
2,atap,/equities/trimitra-prawara-goldland-tbk-pt,1168052,IDR,Millions
3,cash,/equities/cashlez-worldwide-indonesia-tbk,1161015,IDR,Millions
4,cbpe,/equities/pt-citra-buana-prasida-tbk,1198668,IDR,Millions
...,...,...,...,...,...
225,food,/equities/sentra-food,1117859,IDR,Millions
226,glob,/equities/global-telesho,101415,IDR,Millions
227,real,/equities/repower-asia-indonesia-pt,1155912,IDR,Millions
228,asha,/equities/cilacap-samudera-fishing-industry,1191905,IDR,Millions


In [71]:
expected_value = [
    # Income Statement
    "Total Revenue", "Net Income", "Diluted Weighted Average Shares",
    "Gross Profit", "Net Income Before Taxes", "Provision for Income Taxes",
    "Interest Expense (Income) - Net Operating", "Operating Income",
    "Cash & Due from Banks",

    # Balance Sheet
    "Total Assets", "Total Liabilities", "Total Current Liabilities", 
    "Total Equity", "Current Port. of LT Debt/Capital Leases", 
    "Total Long Term Debt", "Total Liabilities & Shareholders' Equity", 
    "Minority Interest", "Cash and Short Term Investments", 
    "Cash & Equivalents", "Total Current Assets",

    # Cash FLow
    "Cash From Operating Activities", "Free Cash Flow"
]

In [72]:
url_currency = 'https://raw.githubusercontent.com/supertypeai/sectors_get_conversion_rate/master/conversion_rate.json'
response = requests.get(url_currency)
data = response.json()
rate = float(data['USD']['IDR'])

# Scraper Qquarterly

In [91]:
data_list = []

def get_values(soup, data_dict):
    tbodies = soup.find("tbody")
    rows = tbodies.find_all("tr")
    for row in rows:
        td = row.find_all("td")
        key = td[0].find("span").get_text()
        values = [t.get_text() if t.get_text() != "-" else np.nan for t in td[1:]]
        if len(values) > 4:
            continue
        # if key == "Diluted Weighted Average Shares":
        #     data_dict[key] = [float(value) for value in values]
        #     continue
        try:
            if currency == "USD":
                if fold == "Billions":
                    values = [float(value) * rate * 1000000000 for value in values]
                else:
                    values = [float(value) * rate * 1000000 for value in values]
            else:
                if fold == "Billions":
                    values = [float(value) * 1000000000 for value in values]
                else:
                    values = [float(value) * 1000000 for value in values]
        except Exception as e:
            print(f"error di {symbol}, dengan error: {e}")
        # print("values: ", values)
        if key in expected_value:
            data_dict[key] = values

for index, row in data_link.iterrows():
# for symbol, link in zip(data_link["symbols"].tolist()[:5], data_link["href"].tolist()[:5]):
    # Income Statement
    if index == 4:
        break
    symbol = row["symbols"]
    link = row["href"]
    currency = row["currency"]
    fold = row["fold"]
    is_url = f"https://www.investing.com{link}-income-statement"
    html_content = requests.get(is_url).text
    soup = BeautifulSoup(html_content, "html.parser")
    period = soup.find("tr", class_ = "alignBottom")
    quarters = period.find_all("th")
    data_dict = {"symbol" : [], "date" : []}
    for quarter in quarters:
        try:
            year = quarter.find("span").get_text()
            day, month = quarter.find("div").get_text().split("/")
            date = str(year) + "-" + str(month) + "-" + str(day)
            data_dict["symbol"].append(symbol.upper() + ".JK")
            data_dict["date"].append(date)
        except:
            pass
    get_values(soup, data_dict)
    # Balance Sheet
    bs_url = f"https://www.investing.com{link}-balance-sheet"
    html_content = requests.get(bs_url).text
    soup = BeautifulSoup(html_content, "html.parser")
    get_values(soup, data_dict)
    # Cash Flow
    bs_url = f"https://www.investing.com{link}-cash-flow"
    html_content = requests.get(bs_url).text
    soup = BeautifulSoup(html_content, "html.parser")
    get_values(soup, data_dict)
    not_avail = [na for na in expected_value if na not in data_dict.keys()]
    for na in not_avail:
        data_dict[na] = [np.nan, np.nan, np.nan, np.nan]
    data_list.append(pd.DataFrame(data_dict))

In [93]:
def fe_and_rename(data_list):
    df = pd.concat(data_list)
    df["total_debt"] = df["Current Port. of LT Debt/Capital Leases"] + df["Total Long Term Debt"]
    df["stackholders_equity"] = df["Total Liabilities & Shareholders' Equity"] - df["Total Liabilities"] - df["Minority Interest"]
    df["ebit"] = df["Net Income Before Taxes"] - df["Interest Expense (Income) - Net Operating"]
    df["total_non_current_assets"] = df["Total Assets"] - df["Total Current Assets"]
    df["ebitda"] = np.nan
    columns_rename = {
        "Cash From Operating Activities" : "net_operating_cash_flow",
        "Total Assets" : "total_assets",
        "Total Liabilities" : "total_liabilities",
        "Total Current Liabilities" : "total_current_liabilities",
        "Total Equity" : "total_equity",
        "Total Revenue" : "total_revenue",
        "Net Income" : "net_income",
        # "total_debt",
        # "stockholders_equity",
        # "ebit",
        # "ebitda",
        # "total_non_current_assets",
        "Cash and Short Term Investments" : "cash_and_short_term_investments",
        "Cash & Equivalents" : "cash_only",
        "Cash & Due from Banks" : "total_cash_and_due_from_banks",
        "Diluted Weighted Average Shares" : "diluted_shares_outstanding",
        "Gross Profit" : "gross_income",
        "Net Income Before Taxes" : "pretax_income",
        "Provision for Income Taxes" : "income_taxes",
        "Free Cash Flow" : "free_cash_flow",
        "Interest Expense (Income) - Net Operating" : "interest_expense_non_operating",
        "Operating Income" : "operating_income"
    }
    data = df.rename(columns=columns_rename).drop(['Minority Interest', 'Current Port. of LT Debt/Capital Leases', 'Total Long Term Debt', "Total Liabilities & Shareholders' Equity", "Total Current Assets"], axis = 1)
    return data

In [95]:
data = fe_and_rename(data_list)

# Scraper Annual

In [97]:
data_list = []
for index, row in data_link.iterrows():
    # Income Statement
    if index == 4:
        break
    symbol = row["symbols"]
    link = row["href"]
    currency = row["currency"]
    fold = row["fold"]
    id = row["id"]
    is_url = f"https://www.investing.com/instruments/Financials/changereporttypeajax?action=change_report_type&pair_ID={id}&report_type=INC&period_type=Annual"
    html_content = requests.get(is_url).text
    soup = BeautifulSoup(html_content, "html.parser")
    periods = soup.find_all("th")
    data_dict = {
        "symbol" : [],
        "date" : []
    }
    dates = []
    for index, period in enumerate(periods[1:]):
        data_dict["symbol"].append(symbol.upper() + ".JK")
        if period.get_text() == "":
            date = datetime.strptime(data_dict["date"][-1], '%Y-%m-%d').date()
            date = str(date.replace(year=date.year-1))
            data_dict["date"].append(date)
            continue
        try:
            year = period.find("span").get_text()
            day, month = period.find("div").get_text().split("/")
            date = str(year) + "-" + str(month) + "-" + str(day)
            data_dict["date"].append(date)
        except Exception as e:
            print(f"error pada {symbol}: {e}")
            pass
    get_values(soup, data_dict)
    bs_url = f"https://www.investing.com/instruments/Financials/changereporttypeajax?action=change_report_type&pair_ID={id}&report_type=BAL&period_type=Annual"
    html_content = requests.get(bs_url).text
    soup = BeautifulSoup(html_content, "html.parser")
    get_values(soup, data_dict)
    cf_url = f"https://www.investing.com/instruments/Financials/changereporttypeajax?action=change_report_type&pair_ID={id}&report_type=CAS&period_type=Annual"
    html_content = requests.get(cf_url).text
    soup = BeautifulSoup(html_content, "html.parser")
    get_values(soup, data_dict)
    data_list.append(pd.DataFrame(data_dict))
pd.concat(data_list).reset_index(drop = True)

,symbol,date,Total Revenue,Gross Profit,Interest Expense (Income) - Net Operating,Operating Income,Net Income Before Taxes,Provision for Income Taxes,Minority Interest,Net Income,...,Cash & Equivalents,Total Assets,Total Current Liabilities,Current Port. of LT Debt/Capital Leases,Total Liabilities,Total Long Term Debt,Total Equity,Total Liabilities & Shareholders' Equity,Cash From Operating Activities,Free Cash Flow
0,CBRE.JK,2023-12-31,4.272817e+10,1.512657e+10,-1.929560e+09,3.079280e+09,1.417180e+09,5.127400e+08,NaN,9.044500e+08,...,1.735855e+11,3.529656e+11,3.004078e+10,1.836360e+10,1.842827e+11,1.540195e+11,1.686829e+11,3.529656e+11,3.980970e+09,-4.942555e+10
1,CBRE.JK,2022-12-31,3.295854e+10,1.487855e+10,-8.371800e+08,9.504870e+09,7.894290e+09,3.955000e+08,NaN,7.498790e+09,...,2.526390e+09,1.299273e+11,1.493470e+10,4.570340e+09,3.860205e+10,2.358565e+10,9.132528e+10,1.299273e+11,6.086260e+09,1.315997e+10
2,CBRE.JK,2021-12-31,NaN,NaN,NaN,NaN,NaN,7.503900e+08,NaN,NaN,...,1.640860e+09,NaN,8.583039e+10,NaN,NaN,NaN,1.329540e+09,NaN,NaN,NaN
3,CBRE.JK,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LAJU.JK,2023-12-31,1.667850e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,LAJU.JK,2022-12-31,1.479179e+11,3.688377e+10,-7.759160e+09,2.386182e+10,1.555104e+10,3.327210e+09,NaN,1.222382e+10,...,1.479563e+10,1.202372e+11,3.661156e+10,2.036018e+10,6.427827e+10,2.691881e+10,NaN,1.202372e+11,2.027756e+10,1.360587e+10
6,LAJU.JK,2021-12-31,1.179755e+11,2.735127e+10,-7.067890e+09,1.753683e+10,1.076254e+10,2.415000e+09,NaN,8.347540e+09,...,1.000930e+09,1.049701e+11,2.433634e+10,1.168142e+10,6.157852e+10,3.636016e+10,4.339154e+10,1.049701e+11,2.027202e+10,1.673163e+10
7,LAJU.JK,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ATAP.JK,2023-12-31,2.365346e+10,8.944550e+09,-1.798960e+09,2.928060e+09,1.123970e+09,9.611400e+08,NaN,1.628200e+08,...,1.096235e+10,1.137727e+11,1.927705e+10,NaN,3.418235e+10,1.463725e+10,7.959031e+10,1.137727e+11,-3.434170e+09,-3.176580e+09
9,ATAP.JK,2022-12-31,4.251911e+10,1.375487e+10,-2.125250e+09,5.458550e+09,3.655040e+09,1.502680e+09,NaN,2.152360e+09,...,1.457382e+10,1.154284e+11,2.046509e+10,NaN,3.581013e+10,1.499595e+10,7.961828e+10,1.154284e+11,-5.648190e+09,-7.560490e+09
